In [1]:
import pyodbc
import pandas as pd
import pandas.io.sql as psql
import time

In [2]:
server = r'130.159.75.36'
database = r'NMIS'
username = r'Ignition_DB_Reader'
password = r'NMISIgnitionDB'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+ ';ENCRYPT=no; UID='+username+';PWD='+ password)

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [ ]:
#cursor = cnxn.cursor()

In [ ]:
#cursor.execute("""SELECT TOP (10000) * FROM dbo.CMI_DB""")

In [ ]:
#selected = cursor.fetchall()
#selected

In [ ]:
#start = time.time()

#rows = cursor.execute("SELECT TOP (1000000) * FROM dbo.CMI_DB").fetchall() 

In [ ]:
#Read cursor data into Pandas dataframe
#df = pd.DataFrame([tuple(t) for t in rows])

In [ ]:
#df

In [ ]:
#end = time.time()
#print(end - start)

In [ ]:
#start = time.time()
#rows_2 = cursor.execute("SELECT TOP (1000000) * FROM dbo.CMI_DB").fetchall() 

In [ ]:
#Read cursor data into Pandas dataframe
#df_2 = pd.DataFrame((tuple(t) for t in rows_2))

In [ ]:
#df_2

In [ ]:
#end = time.time()
#print(end - start)

In [ ]:
start = time.time()
df_all = pd.read_sql("SELECT * FROM dbo.CMI_DB where t_stamp between '2022-11-21' and '2022-11-26';", cnxn)
df_all

In [ ]:
end = time.time()
print(end - start)

In [ ]:
import sqlite3
import pymssql
start = time.time()

In [ ]:
df_all.to_csv('df_all.csv')

In [ ]:
df_3 = pd.read_sql_query("SELECT * FROM dbo.CMI_DB;", cnxn)
df_3

In [ ]:
end = time.time()
print(end - start)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_all.plot(x='t_stamp', y='FURNACE_0126_341_04_O2', kind='line')
plt.show()


In [ ]:
df_4 = df_3[(df_3['t_stamp'] > '2022-10-01 00:00:00.000') & (df_3['t_stamp'] <= '2022-11-15 00:00:00.000')]

In [ ]:
df_4.plot(x='t_stamp', y='FURNACE_0126_341_04_O2', kind='line')
plt.show()

In [ ]:
df_4.info()

In [ ]:
df_5 = df_3[(df_3['t_stamp'] > '2022-10-27 00:00:00.000') & (df_3['t_stamp'] <= '2022-10-30 00:00:00.000')]

In [ ]:
df_5.plot(x='t_stamp', y='FURNACE_0126_341_04_O2', kind='scatter')
plt.show()

In [ ]:
df_5['t_stamp'] = pd.to_datetime(df_5['t_stamp'], format='%Y-%m-%d')
df_5.set_index('t_stamp').plot(subplots=True, layout=(12,4), figsize=(30,50))

In [ ]:
def find_plateaus(F, min_length=200, tolerance = 0.75, smoothing=25):
    '''
    Finds plateaus of signal using second derivative of F.

    Parameters
    ----------
    F : Signal.
    min_length: Minimum length of plateau.
    tolerance: Number between 0 and 1 indicating how tolerant
        the requirement of constant slope of the plateau is.
    smoothing: Size of uniform filter 1D applied to F and its derivatives.
    
    Returns
    -------
    plateaus: array of plateau left and right edges pairs
    dF: (smoothed) derivative of F
    d2F: (smoothed) Second Derivative of F
    '''
    import numpy as np
    from scipy.ndimage import uniform_filter1d
    
    # calculate smooth gradients
    smoothF = uniform_filter1d(F, size = smoothing)
    dF = uniform_filter1d(np.gradient(smoothF),size = smoothing)
    d2F = uniform_filter1d(np.gradient(dF),size = smoothing)
    
    def zero_runs(x):
        '''
        Helper function for finding sequences of 0s in a signal
        https://stackoverflow.com/questions/24885092/finding-the-consecutive-zeros-in-a-numpy-array/24892274#24892274
        '''
        iszero = np.concatenate(([0], np.equal(x, 0).view(np.int8), [0]))
        absdiff = np.abs(np.diff(iszero))
        ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
        return ranges
    
    # Find ranges where second derivative is zero
    # Values under eps are assumed to be zero.
    eps = np.quantile(abs(d2F),tolerance) 
    smalld2F = (abs(d2F) <= eps)
    
    # Find repititions in the mask "smalld2F" (i.e. ranges where d2F is constantly zero)
    p = zero_runs(np.diff(smalld2F))
    
    # np.diff(p) gives the length of each range found.
    # only accept plateaus of min_length
    plateaus = p[(np.diff(p) > min_length).flatten()]
    
    return (plateaus, dF, d2F)

In [ ]:
df_all.plot(x='t_stamp', y='ROOF_0104_300_06_TC', kind='line')
plt.show()

In [ ]:
find_plateaus(df_all['ROOF_0104_300_06_TC'], min_length=2000, tolerance = 0.8, smoothing=25)